Generate questions for AWS SA exam

In [2]:
import requests
from dotenv import load_dotenv
import os

In [3]:
load_dotenv() 
GENERATIVE_URI = os.environ['GENERATIVE_URI']

In [4]:
TOTAL_QUESTIONS = 65
DOMAINS = [
    {
        "name": "Secure",
        "percent": 30,
        "num": 19
    },
    {
        "name": "Resilient",
        "percent": 26,
        "num": 16
    },
    {
        "name": "High-Performing",
        "percent": 24,
        "num": 15
    },
    {
        "name": "Cost-Optimized",
        "percent": 20,
        "num": 15 
    }
]

In [5]:
#determin how many questions in each domain
def get_questions_each_domain():
    
    for domain in DOMAINS:
        no_quest = int(domain['percent'] * TOTAL_QUESTIONS / 100)
        print(no_quest)
#test
#get_questions_each_domain()

In [6]:
#Send a POST request
def post_request_generative_ai(text_prompt):
    HEADER = {'Content-Type': 'application/json'}
    json_data = {
        "contents": [
            { "parts": [
                {"text": text_prompt + " Please provide a response in a structured JSON format."}]
            }
        ]
    }

    try:
        r = requests.post(GENERATIVE_URI, headers=HEADER, json=json_data)
        return r.json()
    except Exception as e:
       print(e)
       return {'error': e}

In [7]:
def generate_list_of_questions():
    for domain in DOMAINS:
        domain['num'] = 1 #testing
        text_prompt = "Generate "+str(domain['num'])+" questions for me to practice AWS certified solution architect - associate exam in the domain: Design "+domain['name']+" architecture. I want questions that focus more on real-world scenarios and problem-solving rather than just definitions of services."
        questions = post_request_generative_ai(text_prompt)
        print(questions)
        
#test
generate_list_of_questions()

{'candidates': [{'content': {'parts': [{'text': '```json\n{\n  "question": {\n    "scenario": "You are designing a highly available and secure e-commerce application on AWS. The application consists of a web application tier, an application tier, and a database tier.  Sensitive customer data (credit card numbers, addresses, etc.) is stored in the database.  The application needs to comply with PCI DSS standards.  Your team has decided to use Amazon RDS for PostgreSQL as the database and Amazon EC2 for the application tiers.  You need to minimize the attack surface and ensure data is protected both in transit and at rest.",\n    "question_text": "Describe a secure architecture for this e-commerce application, focusing on the security considerations for each tier.  Specifically address how you would handle encryption (both in transit and at rest), access control, and logging.  Consider factors like IAM roles, security groups, VPC configuration, and any relevant AWS managed services that 